In [3]:
import findspark
findspark.init()
import pandas as pd
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("pysparkdf").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/14 08:11:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
#1. Crear el DataFrame con datos de personas, que incluya nombre, edad y ciudad de residencia

data = [("Alice", 25, "New York"),
        ("Bob", 30, "Los Angeles"),
       ("Charlie", 22, "Chicago")]

In [45]:
df = spark.createDataFrame(data, ["Nombre", "Edad", "Ciudad de Residencia"])
df.show()

+-------+----+--------------------+
| Nombre|Edad|Ciudad de Residencia|
+-------+----+--------------------+
|  Alice|  25|            New York|
|    Bob|  30|         Los Angeles|
|Charlie|  22|             Chicago|
+-------+----+--------------------+



In [47]:
#2. Mostrar solo los nombres de las personas del DataFrame

df.select("Nombre").show()

#3. Filtrar personas cuya edad sea mayor o igual a 25 

df.filter(df["Edad"] > 25).show()

+-------+
| Nombre|
+-------+
|  Alice|
|    Bob|
|Charlie|
+-------+

+------+----+--------------------+
|Nombre|Edad|Ciudad de Residencia|
+------+----+--------------------+
|   Bob|  30|         Los Angeles|
+------+----+--------------------+



In [7]:
#4. Agregar una nueva columna "Pais" con un valor constante para todas las filas

from pyspark.sql.functions import when
df1 = df.withColumn('Pais', when(df.Edad > 0, "USA"))

df1.show()

+-------+----+--------------------+----+
| Nombre|Edad|Ciudad de Residencia|Pais|
+-------+----+--------------------+----+
|  Alice|  25|            New York| USA|
|    Bob|  30|         Los Angeles| USA|
|Charlie|  22|             Chicago| USA|
+-------+----+--------------------+----+



In [8]:
#5. Calcular el promedio de edad de todas las personas

from pyspark.sql.functions import avg

df.agg(avg("Edad").alias("Promedio Edad")).show()

+------------------+
|     Promedio Edad|
+------------------+
|25.666666666666668|
+------------------+



In [9]:
#6. Ordenar el DataFrame por edad en orden descendente

df.sort(df.Edad.desc()).show()

+-------+----+--------------------+
| Nombre|Edad|Ciudad de Residencia|
+-------+----+--------------------+
|    Bob|  30|         Los Angeles|
|  Alice|  25|            New York|
|Charlie|  22|             Chicago|
+-------+----+--------------------+



In [10]:
#7. Agrupar por ciudad y calcular la cantidad de personas en cada ciudad

from pyspark.sql.functions import *
df.groupBy("Ciudad de Residencia").agg(
        count("Nombre").alias("Numero_personas")
        ).show()

+--------------------+---------------+
|Ciudad de Residencia|Numero_personas|
+--------------------+---------------+
|            New York|              1|
|         Los Angeles|              1|
|             Chicago|              1|
+--------------------+---------------+



In [11]:
#8. Renombrar la columna "Nombre" a "Nombre Completo"

df.withColumnRenamed("Nombre", "Nombre Completo").show()

+---------------+----+--------------------+
|Nombre Completo|Edad|Ciudad de Residencia|
+---------------+----+--------------------+
|          Alice|  25|            New York|
|            Bob|  30|         Los Angeles|
|        Charlie|  22|             Chicago|
+---------------+----+--------------------+



In [12]:
#9. Eliminar la columna "Edad" del DaraFrame

df3 = df.drop("Edad")
df3.show()

+-------+--------------------+
| Nombre|Ciudad de Residencia|
+-------+--------------------+
|  Alice|            New York|
|    Bob|         Los Angeles|
|Charlie|             Chicago|
+-------+--------------------+



In [13]:
#10. Realizar una consulta SQL en el DataFrame para seleccionar personas mayores de 20 anios

from pyspark.sql.functions import col, lit

df.filter( (col("Edad") > 20)).show()

+-------+----+--------------------+
| Nombre|Edad|Ciudad de Residencia|
+-------+----+--------------------+
|  Alice|  25|            New York|
|    Bob|  30|         Los Angeles|
|Charlie|  22|             Chicago|
+-------+----+--------------------+



In [14]:
#11. Calcular la suma total de todas las edades

from pyspark.sql.functions import sum

df.agg(sum("Edad").alias("Suma_Edades")).show()

+-----------+
|Suma_Edades|
+-----------+
|         77|
+-----------+



In [15]:
#12. Calcular la edad minima y maxima de todas las personas


from pyspark.sql.functions import min, max

df.agg(min("Edad").alias("Edad_Minima")).show()
df.agg(max("Edad").alias("Edad Maxima")).show()


+-----------+
|Edad_Minima|
+-----------+
|         22|
+-----------+

+-----------+
|Edad Maxima|
+-----------+
|         30|
+-----------+



In [16]:
# 13. Filtrar personas cuya ciudad de residencia sea "Chicago" y edad sea menor de "30"

from pyspark.sql.functions import col, lit

df.filter((col("Ciudad de Residencia") == lit("Chicago"))
         & (col("Edad") < 30)).show()

+-------+----+--------------------+
| Nombre|Edad|Ciudad de Residencia|
+-------+----+--------------------+
|Charlie|  22|             Chicago|
+-------+----+--------------------+



In [17]:
#14. Agregar una nueva columna "EdadDuplicada" que contenga el doble de la edad

df4 = df.withColumn("EdadDuplicada", (df.Edad * 2)).show()

+-------+----+--------------------+-------------+
| Nombre|Edad|Ciudad de Residencia|EdadDuplicada|
+-------+----+--------------------+-------------+
|  Alice|  25|            New York|           50|
|    Bob|  30|         Los Angeles|           60|
|Charlie|  22|             Chicago|           44|
+-------+----+--------------------+-------------+



In [18]:
# 15. Convertir todas las edades en anios a meses

df5 = df.withColumn("Anios a Meses", (df.Edad * 12)).show()

+-------+----+--------------------+-------------+
| Nombre|Edad|Ciudad de Residencia|Anios a Meses|
+-------+----+--------------------+-------------+
|  Alice|  25|            New York|          300|
|    Bob|  30|         Los Angeles|          360|
|Charlie|  22|             Chicago|          264|
+-------+----+--------------------+-------------+



In [19]:
# 16. Contar el numero total de personas en el DataFrame

df.agg(count("Nombre").alias("Conteo Personas")).show()

+---------------+
|Conteo Personas|
+---------------+
|              3|
+---------------+



In [20]:
# 17. Filtrar personas cuya edad sea un numero par

df.filter( (col("Edad") % 2 == 0 )).show()

+-------+----+--------------------+
| Nombre|Edad|Ciudad de Residencia|
+-------+----+--------------------+
|    Bob|  30|         Los Angeles|
|Charlie|  22|             Chicago|
+-------+----+--------------------+



In [23]:
# 18. Calcular la cantidad de personas por rango de edades( 0-20, 21-40-, 41,60, 61+).
from pyspark.sql.functions import expr

cond = """case when Edad <= 20 then 'de 0 a 20'
                    else case when Edad >20 and Edad <=40 then 'de 21 a 40'
                        else case when Edad >40 and Edad <=60 then 'de 41 a 60'
                            else case when Edad >60 then '60+'
                                else 'rango de edad invalido'
                                    end
                                end
                            end
                        end as rango_edad"""

newdf = df.withColumn("rango de edad", expr(cond))
newdf.show()

+-------+----+--------------------+-------------+
| Nombre|Edad|Ciudad de Residencia|rango de edad|
+-------+----+--------------------+-------------+
|  Alice|  25|            New York|   de 21 a 40|
|    Bob|  30|         Los Angeles|   de 21 a 40|
|Charlie|  22|             Chicago|   de 21 a 40|
+-------+----+--------------------+-------------+



In [24]:
#19. Contar cuantas personas tienen el mismo nombre

from pyspark.sql.functions import col

df.groupBy("Nombre").agg(
    count("Nombre").alias("Numero_de_personas_con_ese_nombre")
).show()

+-------+---------------------------------+
| Nombre|Numero_de_personas_con_ese_nombre|
+-------+---------------------------------+
|  Alice|                                1|
|    Bob|                                1|
|Charlie|                                1|
+-------+---------------------------------+



In [44]:
#20. Concaternar las columnas "Nombre" y "Ciudad" en una nueva columna llamada "InformacionPersonal"

from pyspark.sql.functions import concat, col

df = df.withColumnRenamed("Ciudad de Residencia","ciudad")

df5 = df.select(concat(df.Nombre, df.ciudad ).alias("InformacionPersonal"))
df5.show()

+-------------------+
|InformacionPersonal|
+-------------------+
|      AliceNew York|
|     BobLos Angeles|
|     CharlieChicago|
+-------------------+

